In [1]:
import numpy as np
import sklearn
import pandas as pd
import numpy as np
import random as rnd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import  metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import History
import tensorflow as tf
from tensorflow import keras
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import History
from keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd
import os
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.base import ClassifierMixin
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import class_weight
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import FeatureUnion
from pandas import DataFrame
import warnings
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from keras.layers import Flatten, BatchNormalization
from sklearn.model_selection import RandomizedSearchCV


print(tf.__version__)
print(keras.__version__)

2.9.0
2.9.0


In [2]:
warnings.simplefilter(action='ignore', category=FutureWarning)

DataImport

In [3]:
PATH = os.path.join("data", "bank-additional", "bank-additional")

In [4]:
def load_data(filename, path=PATH):
    csv_path = os.path.join(path, filename)
    return pd.read_csv(csv_path,sep=';')

In [5]:
full_data = load_data("bank-additional-full.csv")

In [6]:
seed=64

In [7]:
cater=['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']
cater_no_def=['job', 'marital', 'education', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']
numer=['age', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
all=['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome','age', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed','y']

In [8]:
class DulicateDeleter(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X,y=None):
        return X.drop_duplicates()

In [9]:
def unknown_number(row):
    bbb=0
    # print(row)
    for sa in row:
        if sa == 'unknown':
            bbb=bbb+1
    return bbb

class UnknownDeleter(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X,y=None):
        return X.drop(X[X.apply(lambda row:unknown_number(row), axis=1)>=3].index)

In [10]:
def null_number(row):
    bbb=0
    # print(row)
    for sa in row:
        if pd.isnull(sa):
            bbb=bbb+1
    return bbb

class NullDeleter(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X,y=None):
        return X.drop(X[X.apply(lambda row:null_number(row), axis=1)>=1].index)

In [11]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X,y=None):
        return X[self.attribute_names]

In [12]:
delete_pip = Pipeline([
        ("delete_null", NullDeleter()),
        ("delete_unknown", UnknownDeleter()),
        ("delete_duplicates", DulicateDeleter())
    ])

In [13]:
delete_pip.fit_transform(full_data)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [14]:
print(full_data.shape)
full_data=delete_pip.fit_transform(full_data)
print(full_data.shape)

(41188, 21)
(40848, 21)


In [15]:
label_encoder = LabelEncoder()
full_data['y'] = label_encoder.fit_transform(full_data['y'])

In [16]:
full_data=delete_pip.fit_transform(full_data)

# _, cut_data = train_test_split(full_data, test_size=0.05, stratify=full_data['y'], random_state=seed)
cut_data=full_data

X = cut_data.drop(['y'], axis=1)
y = cut_data['y'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)
print("X_train.shape: {} y_train.shape: {}".format(X_train.shape, y_train.shape))
print("X_test.shape: {} y_test.shape: {}".format(X_test.shape, y_test.shape))


X_train.shape: (32678, 20) y_train.shape: (32678,)
X_test.shape: (8170, 20) y_test.shape: (8170,)


pipeline definition

In [17]:
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X], index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

In [18]:
num_pip = Pipeline([
        ("select_cat", DataFrameSelector(numer)),
        ("imputer_mean", SimpleImputer(missing_values=np.nan, strategy="median")),
        ("scaler", StandardScaler())
    ])

In [19]:
num_pip.fit_transform(X_train)

array([[-0.57489342,  0.1968458 , -0.35102053, ...,  0.94680367,
         0.77827069,  0.84700494],
       [ 0.86646422,  0.1968458 , -0.35102053, ..., -0.4749137 ,
         0.77366624,  0.84700494],
       [ 0.09774014, -5.05872991,  1.66420239, ..., -1.22885473,
        -1.36337761, -0.93000812],
       ...,
       [-0.47880291,  0.1968458 , -0.35102053, ...,  0.94680367,
         0.77827069,  0.84700494],
       [-0.95925546,  0.1968458 , -0.35102053, ...,  0.94680367,
         0.77711958,  0.84700494],
       [ 2.69218391,  0.1968458 ,  1.66420239, ...,  0.10669795,
        -1.68108518, -2.17529479]])

In [20]:
cat_pip = Pipeline([
        ("select_cat", DataFrameSelector(cater_no_def)),
        ("imputer_freq_unk", SimpleImputer(missing_values="unknown", strategy="most_frequent")),
        ("imputer_freq_nan", SimpleImputer(missing_values=np.nan, strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse=False, handle_unknown = 'ignore'))
    ])

In [21]:
cat_pip.fit_transform(X_train)

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [22]:
default_pip = Pipeline([
        ("select_default", DataFrameSelector(['default'])),
        ("imputer_freq", SimpleImputer(missing_values=np.nan, strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse=False, handle_unknown = 'ignore'))
    ])

In [23]:
default_pip.fit_transform(X_train)

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [24]:
preprocess_pipeline = FeatureUnion(transformer_list=[
        ("num_pip", num_pip),
        ("cat_pip", cat_pip),
        ("def_pip", default_pip)
    ])

In [25]:
DataFrame(preprocess_pipeline.fit_transform(X_train))

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,-0.574893,0.196846,-0.351021,0.842819,-0.223032,0.946804,0.778271,0.847005,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.866464,0.196846,-0.351021,0.842819,0.596012,-0.474914,0.773666,0.847005,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.097740,-5.058730,1.664202,-1.191444,-1.175127,-1.228855,-1.363378,-0.930008,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,-0.670984,0.196846,-0.351021,-1.191444,-1.175127,-1.228855,-1.331722,-0.930008,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.962555,0.196846,-0.351021,-1.191444,-0.860642,-1.422725,-1.267835,-0.930008,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32673,-0.863165,0.196846,-0.351021,-0.746449,2.063034,-2.219749,-1.476187,-2.796561,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
32674,-1.343618,0.196846,-0.351021,-1.890722,-1.054171,-0.065631,-1.353018,-1.245462,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
32675,-0.478803,0.196846,-0.351021,0.842819,-0.223032,0.946804,0.778271,0.847005,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
32676,-0.959255,0.196846,-0.351021,0.842819,-0.223032,0.946804,0.777120,0.847005,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


model learing

In [26]:
labels = {'model_name': [], 'model': [], 'model_history': [], 'model_params': [], 'proba': [], 'precision_score': [], 'recall_score': [], 'f1_score': [], 'accuracy_score': [], 'balanced_accuracy_score': []}

my_models = pd.DataFrame(labels)
added_models=0

In [27]:
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))
print(class_weights_dict)
class_weights_ratio=class_weights_dict[1]/class_weights_dict[0]
print(class_weights_ratio)

{0: 0.5640945969273261, 1: 4.400484783194183}
7.800969566388366


In [28]:
def archive_model(name, model, params=None, history=None, proba=False):
    y_test_pred=model.predict(X_test)
    to_append = {
        'model_name': name,
        'model': model,
        'model_history': history,
        'model_params': params,
        'proba': proba,
        'precision_score': metrics.precision_score(y_test, y_test_pred),
        'recall_score': metrics.recall_score(y_test, y_test_pred),
        'f1_score': metrics.f1_score(y_test, y_test_pred),
        'accuracy_score': metrics.accuracy_score(y_test, y_test_pred),
        'balanced_accuracy_score': metrics.balanced_accuracy_score(y_test, y_test_pred)}
    return to_append

In [29]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

In [30]:
class DumbClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        pass

    def predict(self, X, y=None):
        return np.zeros((len(X)))

    def predict_proba(self, X, y=None):
        pass

In [31]:
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', DumbClassifier())])

param_grid = {}

grid = GridSearchCV(pipe, param_grid, cv=kfold, verbose=1)

grid.fit(X_train, y_train)

print(grid.best_params_)

model_name="{}: Dumb Classifier - {}".format(added_models, None)
added_models=added_models+1
my_models = my_models.append(
    archive_model(model_name,grid.best_estimator_, proba=True, params=grid.best_params_), ignore_index = True)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
{}


C:\Users\wojte\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', LogisticRegression(class_weight=None, solver='liblinear' , verbose=1))])

param_grid = {
            'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
            'classifier__penalty': ['l1', 'l2']
}

grid = GridSearchCV(pipe, param_grid, cv=kfold, verbose=1, scoring='balanced_accuracy')

grid.fit(X_train, y_train)
print(grid.best_params_)

model_name="{}: Logistic Regression - {}".format(added_models, None)
added_models=added_models+1
my_models = my_models.append(
    archive_model(model_name,grid.best_estimator_, proba=True, params=grid.best_params_), ignore_index = True)

Fitting 5 folds for each of 14 candidates, totalling 70 fits
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]{'classifier__C': 1, 'classifier__penalty': 'l2'}


In [33]:
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', LogisticRegression(class_weight='balanced', solver='liblinear' , verbose=1))])

param_grid = {
            'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
            'classifier__penalty': ['l1', 'l2']
}

grid = GridSearchCV(pipe, param_grid, cv=kfold, verbose=1, scoring='balanced_accuracy')

grid.fit(X_train, y_train)
print(grid.best_params_)

model_name="{}: Logistic Regression - {}".format(added_models, 'balanced')
added_models=added_models+1
my_models = my_models.append(
    archive_model(model_name,grid.best_estimator_, proba=True, params=grid.best_params_), ignore_index = True)

Fitting 5 folds for each of 14 candidates, totalling 70 fits
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]{'classifier__C': 0.1, 'classifier__penalty': 'l1'}


In [ ]:

pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', SVC(kernel="rbf", class_weight=None, probability=False, verbose=1))])

param_grid = {
            'classifier__gamma': [10**(-4),10**(-3), 10**(-2), 10**(-1), 1, 10,100],
            'classifier__C': [ 10**(-2), 10**(-1), 1, 10**(1), 10**(2)]
}

grid = GridSearchCV(pipe, param_grid, cv=kfold, verbose=1, scoring='balanced_accuracy')

grid.fit(X_train, y_train)
print(grid.best_params_)

model_name="{}: SVC rbf - {}".format(added_models, None)
added_models=added_models+1
my_models = my_models.append(
archive_model(model_name,grid.best_estimator_, proba=False, params=grid.best_params_), ignore_index = True)

Fitting 5 folds for each of 35 candidates, totalling 175 fits
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][L

In [ ]:
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', SVC(kernel="rbf", class_weight='balanced', probability=False, verbose=1))])

param_grid = {
            'classifier__gamma': [10**(-4),10**(-3), 10**(-2), 10**(-1), 1,],
            'classifier__C': [10**(-1), 1, 10**(1), 10**(2), 10**(3)]
}

grid = GridSearchCV(pipe, param_grid, cv=kfold, verbose=1, scoring='balanced_accuracy')

grid.fit(X_train, y_train)
print(grid.best_params_)


model_name="{}: SVC rbf - {}".format(added_models, 'balanced')
added_models=added_models+1
my_models = my_models.append(
archive_model(model_name,grid.best_estimator_, proba=False, params=grid.best_params_), ignore_index = True)

In [ ]:
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', SVC(kernel='linear', class_weight='balanced', probability=False, verbose=1))])

param_grid = {
            'classifier__C': [10**(-2), 10**(-1), 1, 10**(1), 10**(2)]
}

grid = GridSearchCV(pipe, param_grid, cv=kfold, verbose=1, scoring='balanced_accuracy')

grid.fit(X_train, y_train)
print(grid.best_params_)


model_name="{}: SVC linear - {}".format(added_models, 'balanced')
added_models=added_models+1
my_models = my_models.append(
archive_model(model_name,grid.best_estimator_, proba=False ,params=grid.best_params_), ignore_index = True)

In [ ]:

pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', SVC(kernel='poly', class_weight='balanced', probability=False, verbose=1))])

param_grid = {
            'classifier__degree': [1,2,3,4],
            'classifier__C': [10**(-2), 10**(-1), 1, 10**(1), 10**(2)],
            'classifier__coef0': [0, 0.5, 1, 2]
}

grid = GridSearchCV(pipe, param_grid, cv=kfold, verbose=1, scoring='balanced_accuracy')

grid.fit(X_train, y_train)
print(grid.best_params_)

model_name="{}: SVC poly - {}".format(added_models, 'balanced')
added_models=added_models+1
my_models = my_models.append(
archive_model(model_name,grid.best_estimator_, proba=False, params=grid.best_params_), ignore_index = True)

In [ ]:

pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', GradientBoostingClassifier( verbose=1))])

param_grid = {
    # "classifier__loss":["deviance"],
    # "classifier__learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    # "classifier__min_samples_split": np.linspace(0.1, 0.5, 12),
    # "classifier__min_samples_leaf": np.linspace(0.1, 0.5, 12),
    # "classifier__max_depth":[3,5,8],
    # "classifier__max_features":["log2","sqrt"],
    # "classifier__criterion": ["friedman_mse",  "mae"],
    # "classifier__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    # "classifier__n_estimators":[10]
    "classifier__loss":["deviance", "exponential"],
    "classifier__learning_rate": [0.01, 0.1, 1],
    "classifier__min_samples_leaf": [0.1, 0.5],
    "classifier__max_features":["log2","sqrt"],
    "classifier__criterion": ["friedman_mse",  "squared_error"],
    "classifier__subsample":[ 0.5, 0.8, 1],
    "classifier__n_estimators":[1, 5, 10]
    }

grid = GridSearchCV(pipe, param_grid, cv=kfold, verbose=1)

grid.fit(X_train, y_train)
print(grid.best_params_)

model_name="{}: Gradient Boosting - {}".format(added_models, None)
added_models=added_models+1
my_models = my_models.append(
archive_model(model_name,grid.best_estimator_, proba=True, params=grid.best_params_), ignore_index = True)

In [ ]:
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', XGBClassifier(scale_pos_weight = class_weights_ratio, verbosity = 2, learning_rate=0.02, n_estimators=600, objective='binary:logistic', nthread=1))])

param_grid = {
        # 'classifier__min_child_weight': [1, 5, 10],
        # 'classifier__gamma': [0.5, 1, 1.5, 2, 5],
        # 'classifier__subsample': [0.6, 0.8, 1.0],
        # 'classifier__colsample_bytree': [0.6, 0.8, 1.0],
        # 'classifier__max_depth': [3, 4, 5]
        'classifier__gamma': [0, 0.5, 1, 5],
        'classifier__subsample': [0.5, 0.8, 1.0],
        'classifier__colsample_bytree': [0.5, 0.8, 1.0],
        'classifier__max_depth': [4, 6, 8]
        }

grid = GridSearchCV(pipe, param_grid, cv=kfold, verbose=1, scoring='balanced_accuracy')

grid.fit(X_train, y_train)
print(grid.best_params_)

model_name="{}: XGBoost - {}".format(added_models, 'balanced')
added_models=added_models+1
my_models = my_models.append(
archive_model(model_name,grid.best_estimator_, proba=True, params=grid.best_params_), ignore_index = True)

In [ ]:
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', DecisionTreeClassifier(class_weight='balanced', verbose=1))])

param_grid = {
    # 'classifier__max_leaf_nodes': list(range(2, 100)),
    # 'classifier__min_samples_split': [2, 3, 4]
    'classifier__max_leaf_nodes': [None, 10, 50],
    'classifier__max_features': ['sqrt', 'log2'],
    'classifier__min_samples_split': [ 2, 3, 4, 5],
    'classifier__criterion': ['gini', 'entropy'],
    'classifier__splitter': ['best', 'random']
}

grid = GridSearchCV(pipe, param_grid, cv=kfold, verbose=1, scoring='balanced_accuracy')

grid.fit(X_train, y_train)
print(grid.best_params_)

model_name="{}: Decision Tree - {}".format(added_models, 'balanced')
added_models=added_models+1
my_models = my_models.append(
archive_model(model_name,grid.best_estimator_, proba=True, params=grid.best_params_), ignore_index = True)

In [ ]:

# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]# Create the random grid
# random_grid = {'classifier__n_estimators': [100,250,500,1000],
#                'classifier__max_features': ['auto', 'sqrt'],
#                'classifier__max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
#                'classifier__min_samples_split': [2, 5, 10],
#                'classifier__min_samples_leaf': [1, 2, 4],
#                'classifier__bootstrap': [True, False],
#                'classifier__criterion': ['gini', 'entropy', 'log_loss'],}

pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', RandomForestClassifier(class_weight='balanced',verbose=1))])

param_grid = {
            'classifier__max_depth': [ 10, 50, 100],
            'classifier__max_features': ['sqrt', 'log2'],
            'classifier__bootstrap': [True, False],
            'classifier__n_estimators': [100, 250, 500, 1000],
}

grid = GridSearchCV(pipe, param_grid, cv = kfold, n_jobs = 2, verbose=1, scoring='balanced_accuracy')
# grid = RandomizedSearchCV(pipe, random_grid, n_iter = 100, cv = kfold, verbose=2, random_state=seed, n_jobs = 2)

grid.fit(X_train, y_train)
print(grid.best_params_)

model_name="{}: Random Forest - {}".format(added_models, 'balanced')
added_models=added_models+1
my_models = my_models.append(
archive_model(model_name,grid.best_estimator_, proba=True, params=grid.best_params_), ignore_index = True)

deep shit

In [ ]:
input_shape=preprocess_pipeline.transform(X_train).shape[1]

In [ ]:
def neural_network_1():
    model = Sequential()

    model.add(Dense(32, input_dim=input_shape, activation='sigmoid'))
    model.add(Dense(16, activation='sigmoid'))
    model.add(Dense(8, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

model = keras.wrappers.scikit_learn.KerasClassifier(neural_network_1, epochs=100, batch_size=64, validation_split=0.2)
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', model)])
history = History()

trained_model = pipe.fit(X_train, y_train, classifier__callbacks=[history])

model_name="{}: NN 1 - {}".format(added_models, None)
added_models=added_models+1
my_models = my_models.append(
archive_model(model_name,trained_model, proba=True, history=history), ignore_index = True)

In [ ]:
def neural_network_1():
    model = Sequential()

    model.add(Dense(32, input_dim=input_shape, activation='sigmoid'))
    model.add(Dense(16, activation='sigmoid'))
    model.add(Dense(8, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

model = keras.wrappers.scikit_learn.KerasClassifier(neural_network_1, epochs=100, batch_size=64, validation_split=0.2)
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', model)])
history = History()

trained_model = pipe.fit(X_train, y_train, classifier__class_weight=class_weights_dict, classifier__callbacks=[history])

model_name="{}: NN 1 - {}".format(added_models, 'balanced')
added_models=added_models+1
my_models = my_models.append(
archive_model(model_name,trained_model, proba=True, history=history), ignore_index = True)

In [ ]:
def neural_network_2():
    model = Sequential()

    model.add(Dense(500, input_dim=input_shape, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])
    return model

model = keras.wrappers.scikit_learn.KerasClassifier(neural_network_2, epochs=100, batch_size=64,  validation_split=0.2)
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', model)])
history = History()

trained_model = pipe.fit(X_train, y_train, classifier__class_weight=class_weights_dict, classifier__callbacks=[history])

model_name="{}: NN 2 - {}".format(added_models, 'balanced')
added_models=added_models+1
my_models = my_models.append(
archive_model(model_name,trained_model, proba=True, history=history), ignore_index = True)

In [ ]:
def neural_network_3():
    model = Sequential()

    model.add(Flatten(input_shape=[input_shape]))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.3))

    model.add(Dense(1000))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.3))

    model.add(Dense(500))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.3))

    model.add(Dense(200))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.3))

    model.add(Dense(1,activation="sigmoid"))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = keras.wrappers.scikit_learn.KerasClassifier(neural_network_3, epochs=1000, batch_size=32,  validation_split=0.2)
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', model)])

history = History()
early_stopping = EarlyStopping(monitor='val_loss', patience=100, mode='min', verbose=1)
trained_model = pipe.fit(X_train, y_train, classifier__class_weight=class_weights_dict, classifier__callbacks=[history,early_stopping])

model_name="{}: NN 3 - {}".format(added_models, 'balanced')
added_models=added_models+1
my_models = my_models.append(
archive_model(model_name,trained_model, proba=True, history=history), ignore_index = True)

In [ ]:
def build_model_SGD(lyrs=[16,8], activation='relu', dropout_rate=0.3, lr=0.01, mom=0.0, nest=False):
    model = Sequential()
    model.add(keras.layers.InputLayer(input_shape=[input_shape]))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout_rate))

    for i in range(0,len(lyrs)):
        model.add(Dense(lyrs[i]))
        model.add(BatchNormalization())
        model.add(Activation(activation))
        model.add(Dropout(dropout_rate))

    model.add(keras.layers.Dense(1, activation='sigmoid'))
    optimizer = keras.optimizers.SGD(learning_rate=lr, momentum=mom, nesterov=nest)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

model = keras.wrappers.scikit_learn.KerasClassifier(build_model_SGD, epochs=100, batch_size=32,  validation_split=0.2)
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', model)])
history = History()

param_distribs = {
    "classifier__lyrs": [[16,8], [64,32,16], [200,100,50]],
    "classifier__activation": ['relu', 'elu', 'sigmoid', 'tanh'],
    "classifier__lr": [0.001, 0.01, 0.1],
    "classifier__dropout_rate": [0.2, 0.3, 0.4],
    "classifier__mom": [0.0, 0.5, 0.9],
    "classifier__nest": [True, False],
    "classifier__batch_size": [16, 32, 64]
}

early_stopping = EarlyStopping(monitor='val_loss', patience=40, mode='min', verbose=1)
grid = RandomizedSearchCV(pipe, param_distribs, n_iter=50, cv=kfold, verbose=1, scoring='balanced_accuracy')
grid.fit(X_train, y_train, classifier__callbacks=[early_stopping])

print(grid.best_params_)

model_name="{}: NN SGD RS - {}".format(added_models, None)
added_models=added_models+1
my_models = my_models.append(
archive_model(model_name,grid.best_estimator_, proba=True, params=grid.best_params_), ignore_index = True)

In [ ]:
def build_model_SGD(lyrs=[16,8], activation='relu', dropout_rate=0.3, lr=0.01, mom=0.0, nest=False):
    model = Sequential()
    model.add(keras.layers.InputLayer(input_shape=[input_shape]))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout_rate))

    for i in range(0,len(lyrs)):
        model.add(Dense(lyrs[i]))
        model.add(BatchNormalization())
        model.add(Activation(activation))
        model.add(Dropout(dropout_rate))

    model.add(keras.layers.Dense(1, activation='sigmoid'))
    optimizer = keras.optimizers.SGD(learning_rate=lr, momentum=mom, nesterov=nest)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

model = keras.wrappers.scikit_learn.KerasClassifier(build_model_SGD, epochs=100, batch_size=32,  validation_split=0.2)
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', model)])
history = History()

param_distribs = {
    "classifier__lyrs": [[16,8], [64,32,16], [200,100,50]],
    "classifier__activation": ['relu', 'elu', 'sigmoid', 'tanh'],
    "classifier__lr": [0.001, 0.01, 0.1],
    "classifier__dropout_rate": [0.2, 0.3, 0.4],
    "classifier__mom": [0.0, 0.5, 0.9],
    "classifier__nest": [True, False],
    "classifier__batch_size": [16, 32, 64]
}
#     "classifier__optimizer": ['Adam', 'SGD', 'Adagrad']
early_stopping = EarlyStopping(monitor='val_loss', patience=40, mode='min', verbose=1)
grid = RandomizedSearchCV(pipe, param_distribs, n_iter=50, cv=kfold, verbose=1, scoring='balanced_accuracy')
grid.fit(X_train, y_train,classifier__class_weight=class_weights_dict, classifier__callbacks=[early_stopping])

print(grid.best_params_)

model_name="{}: NN SGD RS - {}".format(added_models, 'balanced')
added_models=added_models+1
my_models = my_models.append(
archive_model(model_name,grid.best_estimator_, proba=True, params=grid.best_params_), ignore_index = True)

In [ ]:
def build_model_Adam(lyrs=[16,8], activation='relu', dropout_rate=0.3, lr=0.01):
    model = Sequential()
    model.add(keras.layers.InputLayer(input_shape=[input_shape]))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout_rate))

    for i in range(0,len(lyrs)):
        model.add(Dense(lyrs[i]))
        model.add(BatchNormalization())
        model.add(Activation(activation))
        model.add(Dropout(dropout_rate))

    model.add(keras.layers.Dense(1, activation='sigmoid'))
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

model = keras.wrappers.scikit_learn.KerasClassifier(build_model_Adam, epochs=100, batch_size=32,  validation_split=0.2)
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', model)])
history = History()

param_distribs = {
    "classifier__lyrs": [[16,8], [64,32,16], [200,100,50]],
    "classifier__activation": ['relu', 'sigmoid', 'tanh'],
    "classifier__lr": [0.001, 0.01, 0.1],
    "classifier__dropout_rate": [0.2, 0.3, 0.4],
    "classifier__batch_size": [16, 32, 64]
}

early_stopping = EarlyStopping(monitor='val_loss', patience=40, mode='min', verbose=1)
grid = RandomizedSearchCV(pipe, param_distribs, n_iter=50, cv=kfold, verbose=1, scoring='balanced_accuracy')
grid.fit(X_train, y_train,classifier__class_weight=class_weights_dict, classifier__callbacks=[early_stopping])

print(grid.best_params_)

model_name="{}: NN Adam RS - {}".format(added_models, 'balanced')
added_models=added_models+1
my_models = my_models.append(
archive_model(model_name,grid.best_estimator_, proba=True, params=grid.best_params_), ignore_index = True)

In [ ]:
def build_model_Adagrad(lyrs=[16,8], activation='relu', dropout_rate=0.3, lr=0.01):
    model = Sequential()
    model.add(keras.layers.InputLayer(input_shape=[input_shape]))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout_rate))

    for i in range(0,len(lyrs)):
        model.add(Dense(lyrs[i]))
        model.add(BatchNormalization())
        model.add(Activation(activation))
        model.add(Dropout(dropout_rate))

    model.add(keras.layers.Dense(1, activation='sigmoid'))
    optimizer = keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

model = keras.wrappers.scikit_learn.KerasClassifier(build_model_Adagrad, epochs=100, batch_size=32,  validation_split=0.2)
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', model)])
history = History()

param_distribs = {
    "classifier__lyrs": [[16,8], [64,32,16], [200,100,50]],
    "classifier__activation": ['relu', 'sigmoid', 'tanh'],
    "classifier__lr": [0.001, 0.01, 0.1],
    "classifier__dropout_rate": [0.2, 0.3, 0.4],
    "classifier__batch_size": [16, 32, 64]
}
#     "classifier__optimizer": ['Adam', 'SGD', 'Adagrad']
early_stopping = EarlyStopping(monitor='val_loss', patience=40, mode='min', verbose=1)
grid = RandomizedSearchCV(pipe, param_distribs, n_iter=50, cv=kfold, verbose=1, scoring='balanced_accuracy')
grid.fit(X_train, y_train,classifier__class_weight=class_weights_dict, classifier__callbacks=[early_stopping])

print(grid.best_params_)

model_name="{}: NN Adagrad RS - {}".format(added_models, 'balanced')
added_models=added_models+1
my_models = my_models.append(
archive_model(model_name,grid.best_estimator_, proba=True, params=grid.best_params_), ignore_index = True)

In [ ]:
def show_con_mat(name, model):
    plt.rcParams.update({'font.size': 16})
    y_test_pred = model.predict(X_test)
    cf_matrix_1=metrics.confusion_matrix(y_test, y_test_pred, normalize='true')
    cf_matrix_2=metrics.confusion_matrix(y_test, y_test_pred, normalize=None)

    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(15,4))

    sns.heatmap(cf_matrix_1, annot=True, cmap='Blues', ax=ax1)
    sns.heatmap(cf_matrix_2, annot=True, cmap='Blues', ax=ax2)
    # Normalized Confusion Matrix
    ax1.set_title('{}\n'.format(name))
    ax1.set_xlabel('\nPredicted Values')
    ax1.set_ylabel('Actual Values ')
    # Confusion Matrix
    ax2.set_title('{}\n'.format(name))
    ax2.set_xlabel('\nPredicted Values')
    ax2.set_ylabel('Actual Values ')

    ax1.xaxis.set_ticklabels(['False', 'True'])
    ax1.yaxis.set_ticklabels(['False', 'True'])

    ax2.xaxis.set_ticklabels(['False', 'True'])
    ax2.yaxis.set_ticklabels(['False', 'True'])

def show_curves(name, model, proba=False):
    plt.rcParams.update({'font.size': 16})
    if proba:
        y_test_pred_proba = model.predict_proba(X_test)[:, 1]
    else:
        y_test_pred_proba = model.predict(X_test)

    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(15,4))
    fpr, tpr, _ = metrics.roc_curve(y_test, y_test_pred_proba)
    # Roc Curve
    ax1.plot(fpr, tpr)
    ax1.set_title('{}\n'.format(name))
    ax1.set_ylabel('True Positive Rate')
    ax1.set_xlabel('False Positive Rate')

    precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_test_pred_proba)

    ax2.plot(recall, precision, color='purple')
    # Precision-Recall Curve
    ax2.set_title('{}\n'.format(name))
    ax2.set_ylabel('Precision')
    ax2.set_xlabel('Recall')

def show_history(name, history):
    plt.rcParams.update({'font.size': 16})
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.title('History for {}\n\n'.format(name))
    plt.gca().set_ylim(0, 1)
    plt.show()

def plot_bars(df,parametr):
    plt.rcParams.update({'font.size': 16})
    print(parametr)

    height = df[parametr]
    bars = df['model_name']
    y_pos = np.arange(len(bars))

    plt.bar(y_pos, height)
    ax = plt.gca()
    ax.set_ylim([0, 1])

    plt.xticks(y_pos, bars, rotation=90)
    plt.rcParams["figure.figsize"] = (30,10)
    plt.rcParams.update({'font.size': 22})

    plt.show()
def make_bars_2(df):

    plt.rcParams["figure.figsize"] = (30,10)
    plt.rcParams.update({'font.size': 22})
    labels = df['model_name']

    bars1 = df['precision_score']
    bars2 = df['recall_score']
    bars3 = df['f1_score']
    bars4 = df['accuracy_score']
    bars5 = df['balanced_accuracy_score']

    width = 0.15
    x = np.arange(len(labels))


    fig, ax = plt.subplots()
    rects1 = ax.bar(x - width*2, bars1, width, label='precision_score')
    rects2 = ax.bar(x - width*1, bars2, width, label='recall_score')
    rects3 = ax.bar(x, bars3, width, label='f1_score')
    rects4 = ax.bar(x + width*1, bars4, width, label='accuracy_score')
    rects5 = ax.bar(x + width*2, bars5, width, label='balanced_accuracy_score')

    ax.set_title('Models parameter breakdown')
    ax.set_xticks(x, labels, rotation=90)
    ax.legend()

    # ax.bar_label(rects1, padding=3)
    # ax.bar_label(rects2, padding=3)
    # ax.bar_label(rects3, padding=3)
    # ax.bar_label(rects4, padding=3)
    # ax.bar_label(rects5, padding=3)

    fig.tight_layout()

    plt.show()

yh

In [ ]:
my_models

In [ ]:
my_models.drop(['model','model_params', 'model_history', 'proba'], axis=1)

In [ ]:
make_bars_2(my_models)

In [ ]:
plot_bars(my_models,'precision_score')
plot_bars(my_models,'recall_score')
plot_bars(my_models,'f1_score')
plot_bars(my_models,'accuracy_score')
plot_bars(my_models,'balanced_accuracy_score')

In [ ]:
for index, row in my_models.iterrows():
    show_con_mat(row['model_name'], row['model'])

In [ ]:
for index, row in my_models.iterrows():
    show_curves(row['model_name'], row['model'], row['proba'])

In [ ]:
for index, row in my_models.iterrows():
    if row['model_history'] is not None:
        show_history(row['model_name'], row['model_history'])